In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

# Load your dataset
filename = "NF-BOT-IOT"
train_data = pd.read_csv('/content/drive/MyDrive/dataset_wireless/NF-BOT-IOT_train_preprocessed.csv', sep=',', encoding='utf-8')
test_data = pd.read_csv('/content/drive/MyDrive/dataset_wireless/NF-BOT-IOT_test_preprocessed.csv', sep=',', encoding='utf-8')

# Prepare training data
X_train = train_data.drop(columns=['label'], axis=1)
y_train = train_data['label']

# Prepare test data
X_test = test_data.drop(columns=['label'], axis=1)
y_test = test_data['label']

# Split a small portion of training and test data for optimization (MBOA)
X_t, _, y_t, _ = train_test_split(X_train, y_train, train_size=0.01, random_state=7)
X_test_t, _, y_test_t, _ = train_test_split(X_test, y_test, train_size=0.01, random_state=7)

# Define custom Monarch Butterfly Optimization Algorithm (MBOA) for feature selection
class MBOAFeatureSelection:
    def __init__(self, num_butterflies=5, max_iter=5):
        self.num_butterflies = num_butterflies
        self.max_iter = max_iter

    def fit(self, X, y):
        start_time = time.time()  # Start timer
        num_features = X.shape[1]
        butterflies = np.random.randint(0, 2, size=(self.num_butterflies, num_features))  # Initialize with binary values
        best_feature_set = None
        best_accuracy = 0.0

        for iteration in range(self.max_iter):
            fitness_values = []

            for butterfly in butterflies:
                selected_features = np.where(butterfly > 0)[0]  # Select features where value > 0

                # Evaluate subset using a simple model (e.g., Random Forest)
                clf = RandomForestClassifier()
                X_subset = X.iloc[:, selected_features]
                clf.fit(X_subset, y)
                accuracy = accuracy_score(y, clf.predict(X_subset))

                fitness_values.append((selected_features, accuracy))

                # Update best solution found
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_feature_set = selected_features

            # Update butterflies based on fitness values
            sorted_indices = np.argsort([fitness[1] for fitness in fitness_values])[::-1]
            butterflies = np.array([butterflies[idx] for idx in sorted_indices])

        end_time = time.time()  # End timer
        execution_time = end_time - start_time  # Calculate execution time

        self.selected_features = best_feature_set
        self.execution_time = execution_time
        self.num_selected_features = len(best_feature_set)
        return self

# Instantiate and run the MBOA algorithm for feature selection on training data
mboa = MBOAFeatureSelection(num_butterflies=5, max_iter=10)
mboa.fit(X_t, y_t)

# Apply selected features to training data
selected_features_train = X_train.columns[mboa.selected_features]

# Create a DataFrame for optimization results
optimization_results = pd.DataFrame({
    "Optimization": ["MBOA"],
    "Execution Time of Optimizer": [mboa.execution_time],
    "No of Feature Selected": [mboa.num_selected_features],
    "Selected Feature": [', '.join(selected_features_train.tolist())]
})

# Save optimization results to a CSV file
optimization_results.to_csv('Monarch_Butterfly_optimization_results.csv', index=False)
